In [16]:
import pandas as pd
from pathlib import Path

PATH = Path.cwd().parent.joinpath('data')

# Goal

The goal of this notebook is to explore if we can use commodity trader's data for our analysis of the shadowfleet. Do they use vessels of the shadowfleet and how have they responded to the war in Ukraine? 

In [17]:
dfs = []
for file in PATH.joinpath('bloomberg', 'charters').glob('*.xlsx'):

    file_extension = file.suffix.lower()[1:]

    if file_extension == 'xlsx':
        df = pd.read_excel(file, engine='openpyxl', skiprows=1)
    elif file_extension == 'xls':
        df = pd.read_excel(file, skiprows=1)
    
    dfs.append(df)

fixtures = pd.concat(dfs)
fixtures.columns = fixtures.columns.str.lower()
fixtures.charterer = 'Vitol'

fixtures.dropna(subset=['imo_number'], inplace=True)
fixtures.drop_duplicates(inplace=True)
fixtures.reset_index(drop=True, inplace=True)
fixtures[['fixture_date', 'loading_date']] = fixtures[['fixture_date', 'loading_date']].apply(pd.to_datetime, errors='coerce')
fixtures.to_csv(PATH.joinpath('bloomberg', 'fixtures.csv'), index=False)

len(fixtures)

15102

In [18]:
fixtures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15102 entries, 0 to 15101
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   ship_name                        15102 non-null  object        
 1   imo_number                       15102 non-null  int64         
 2   fixture_date                     15102 non-null  datetime64[ns]
 3   loading_date                     15102 non-null  datetime64[ns]
 4   normalized_origin_region         15102 non-null  object        
 5   normalized_destination_region_1  15102 non-null  object        
 6   cargo_product                    15102 non-null  object        
 7   charterer                        15102 non-null  object        
 8   dwt                              15102 non-null  int64         
 9   imo_der                          15102 non-null  int64         
 10  map_der                          15102 non-null  object   

In [19]:
kse = pd.read_csv(PATH.joinpath('processed', 'kse_shadowfleet.csv'))
len(kse)

739

In [23]:
# How many vessels are in both datasets?

intersection = set(fixtures[fixtures.fixture_date > '2023-01-01'].imo_number).intersection(kse.imo)
len(intersection)

36

In [24]:
fixtures[fixtures.imo_number.isin(list(intersection))]

,ship_name,imo_number,fixture_date,loading_date,normalized_origin_region,normalized_destination_region_1,cargo_product,charterer,dwt,imo_der,map_der,source
273,SEA HAZEL,9266853,2023-12-13,2023-12-22,South East Asia,Unreported,FUEL OIL,Vitol,106085,9266853,IMO 9266853,Gibson
285,SEA HAZEL,9266853,2023-12-12,2023-12-22,South East Asia,East,DIRTY,Vitol,106085,9266853,IMO 9266853,IHS
330,SEA STAR,9607710,2023-11-28,2023-12-05,East,South East Asia,GASOIL,Vitol,112147,9607710,IMO 9607710,IHS
367,TAURUS II,9402469,2023-11-21,2023-11-30,USGC,UKC,UNKNOWN,Vitol,115459,9402469,IMO 9402469,IHS
420,SIRIUS 1,9285847,2023-11-02,2023-11-11,East Mediterranean,UKC,UNKNOWN,Vitol,115340,9285847,IMO 9285847,IHS
...,...,...,...,...,...,...,...,...,...,...,...,...
14867,BEKS OCEAN,9280586,2021-09-30,2021-10-06,Arabian Gulf,Arabian Gulf,UNL MOGAS,Vitol,72664,9280586,IMO 9280586,Galbraiths
14884,BEKS OCEAN,9280586,2021-09-29,2021-10-06,Arabian Gulf,Arabian Gulf,UNL MOGAS,Vitol,72664,9280586,IMO 9280586,SSY
14963,SINO PROSPERITY,9328132,2021-09-17,2021-09-25,China,South East Asia,CLEAN,Vitol,53160,9328132,IMO 9328132,Galbraiths
14988,SINO PROSPERITY,9328132,2021-09-15,2021-09-25,China,South Korea,JET,Vitol,53160,9328132,IMO 9328132,SSY
